# Limpieza y preparación de datos

## 2.1 Tratamiento de los datos que faltan

### En muchas aplicaciones de análisis de datos es habitual que falten datos. Uno de los objetivos de pandas es hacer que trabajar con datos perdidos sea lo menos doloroso posible. Por ejemplo, todas las estadísticas descriptivas de los objetos de pandas excluyen por defecto los datos que faltan.

### La forma en que se representan los datos que faltan en los objetos de pandas es algo imperfecta, pero es suficiente para la mayoría de los usos en el mundo real. Para datos con dtype float64, pandas utiliza el valor de coma flotante NaN (`Not a Number`) para representar los datos que faltan.

Lo llamamos valor centinela (sentinel value): cuando está presente, indica un valor ausente (o nulo):

In [1]:
import pandas as pd
import numpy as np
float_data = pd.Series([1.8, -3.5, np.nan, 0])

In [2]:
float_data

0    1.8
1   -3.5
2    NaN
3    0.0
dtype: float64

El método `isna` nos da una Serie Booleana con `True` donde los valores son 'nulos':

In [3]:
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

En pandas, hemos adoptado una convención utilizada en el lenguaje de programación R refiriéndonos a los datos perdidos como `NA`, que significa no disponible (`Not Available`). 

El valor `None` (ausencia de un valor o un valor nulo) incorporado en Python también se trata como `NA`:

In [4]:
string_data = pd.Series(['abcd', np.nan, None, 'dfeg'])

string_data

0    abcd
1     NaN
2    None
3    dfeg
dtype: object

In [5]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

El método `isna` nos da una Serie Booleana con `True` donde los valores son 'nulos':

In [6]:
float_data = pd.Series([1, 2, None], dtype='float64')
float_data

0    1.0
1    2.0
2    NaN
dtype: float64

In [7]:
float_data.isna()

0    False
1    False
2     True
dtype: bool

**Otros metodos**


`dropna()`: es un método utilizado para eliminar filas o columnas que contienen valores nulos (NaN).  

`fillna`: Rellena los datos que faltan con algún valor o utilizando un método de interpolación como `ffill` o `bfill`.

`ìsna()`: Devuelve valores booleanos que indican qué valores faltan `NA`. 

`notna`: Negación de `isna`, devuelve `True` para valores no `NA` y False para valores `NA`.

### Filtrar los datos que faltan

Hay algunas formas de filtrar los datos que faltan. Aunque siempre se tiene la opción de hacerlo a mano usando `pandas.isna()` y la indexación booleana, `dropna()` puede ser útil. En una Serie, devuelve la Serie con sólo los datos no nulos y los valores de índice:

In [8]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

Esto es lo mismo que hacer:

In [9]:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

Con los objetos DataFrame, hay diferentes formas de eliminar los datos que faltan. Es posible que desee eliminar las filas o columnas que son todas NA, o sólo las filas o columnas que contienen cualquier NA en absoluto. `dropna()` por defecto elimina cualquier fila que contenga un valor perdido:

In [10]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
             [np.nan, np.nan, np.nan],
             [np.nan, 6.5, 3.]])

data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [11]:
 data.dropna()

,0,1,2
0,1.0,6.5,3.0


Si se pasa `how="all"`, sólo se eliminarán las filas que sean todas NA:

In [12]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Tenga en cuenta que estas funciones devuelven nuevos objetos por defecto y no modifican el contenido del objeto original.

Para soltar (drop) columnas del mismo modo, pase `axis="columns"`:


In [13]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [14]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Supongamos que desea conservar sólo las filas que contengan como máximo un determinado número de observaciones omitidas.  
Puede indicarlo con el argumento `thresh`, el cual especifica el número mínimo de valores no nulos que deben estar presentes en una fila o columna para que no sea eliminada.  
Si una fila o columna no cumple con este umbral mínimo de valores no nulos, será eliminada del DataFrame.

In [15]:
 df = pd.DataFrame(np.random.standard_normal((7, 3)))

In [16]:
df

,0,1,2
0,-0.350195,0.184925,-0.666579
1,-1.603950,0.516483,-0.190964
2,0.064977,0.140970,-0.807149
3,0.205999,0.668086,0.333218
4,0.193293,-1.670206,-0.590758
5,0.496036,-0.706883,0.392845
6,1.310755,1.240568,-1.256120


In [17]:
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan

In [18]:
df

,0,1,2
0,-0.350195,NaN,NaN
1,-1.603950,NaN,NaN
2,0.064977,NaN,-0.807149
3,0.205999,NaN,0.333218
4,0.193293,-1.670206,-0.590758
5,0.496036,-0.706883,0.392845
6,1.310755,1.240568,-1.256120


In [19]:
df.dropna()

,0,1,2
4,0.193293,-1.670206,-0.590758
5,0.496036,-0.706883,0.392845
6,1.310755,1.240568,-1.256120


In [20]:
df.dropna(thresh=2)

,0,1,2
2,0.064977,NaN,-0.807149
3,0.205999,NaN,0.333218
4,0.193293,-1.670206,-0.590758
5,0.496036,-0.706883,0.392845
6,1.310755,1.240568,-1.256120


### Rellenar los datos que faltan

En lugar de filtrar los datos que faltan (y potencialmente descartar otros datos junto con ellos), es posible que desee rellenar los "huecos" de varias maneras. Para la mayoría de los propósitos, el método `fillna()` es la función a utilizar. Llamar a `fillna()` con una constante sustituye los valores que faltan por ese valor:

In [21]:
df

,0,1,2
0,-0.350195,NaN,NaN
1,-1.603950,NaN,NaN
2,0.064977,NaN,-0.807149
3,0.205999,NaN,0.333218
4,0.193293,-1.670206,-0.590758
5,0.496036,-0.706883,0.392845
6,1.310755,1.240568,-1.256120


In [22]:
df.fillna(0)

,0,1,2
0,-0.350195,0.000000,0.000000
1,-1.603950,0.000000,0.000000
2,0.064977,0.000000,-0.807149
3,0.205999,0.000000,0.333218
4,0.193293,-1.670206,-0.590758
5,0.496036,-0.706883,0.392845
6,1.310755,1.240568,-1.256120


Llamando a `fillna()` con un diccionario, puede utilizar un valor de relleno diferente para cada columna:

In [23]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.350195,0.500000,0.000000
1,-1.603950,0.500000,0.000000
2,0.064977,0.500000,-0.807149
3,0.205999,0.500000,0.333218
4,0.193293,-1.670206,-0.590758
5,0.496036,-0.706883,0.392845
6,1.310755,1.240568,-1.256120


Los mismos métodos de interpolación disponibles para la reindexación pueden utilizarse con `fillna()`:

In [24]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

In [25]:
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

,0,1,2
0,-1.722007,-1.895035,-0.090413
1,-1.634339,-1.782433,-0.950678
2,-1.004879,NaN,0.732767
3,1.611951,NaN,-0.385819
4,2.559992,NaN,NaN
5,0.803963,NaN,NaN


In [26]:
 df.fillna(method="ffill")

C:\Users\juanj\AppData\Local\Temp\ipykernel_24584\4205080046.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill")


,0,1,2
0,-1.722007,-1.895035,-0.090413
1,-1.634339,-1.782433,-0.950678
2,-1.004879,-1.782433,0.732767
3,1.611951,-1.782433,-0.385819
4,2.559992,-1.782433,-0.385819
5,0.803963,-1.782433,-0.385819


In [27]:
df.fillna(method="ffill", limit=2)

C:\Users\juanj\AppData\Local\Temp\ipykernel_24584\1627181726.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", limit=2)


,0,1,2
0,-1.722007,-1.895035,-0.090413
1,-1.634339,-1.782433,-0.950678
2,-1.004879,-1.782433,0.732767
3,1.611951,-1.782433,-0.385819
4,2.559992,NaN,-0.385819
5,0.803963,NaN,-0.385819


Con `fillna()` puede hacer muchas otras cosas, como la imputación simple de datos utilizando la mediana o la media estadística:

In [28]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [29]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

**Funciones de argumento para `fillna`**

`value`: Valor escalar u objeto tipo diccionario que se utilizará para rellenar los valores que faltan.

`method`: Método de interpolación: uno de "bfill" (relleno hacia atrás) o "ffill" (relleno hacia delante); por defecto es None

`axis`: Eje de relleno ("index" o "columns"); por defecto axis="index".

`limit`: Para el llenado hacia delante y hacia atrás, número máximo de periodos consecutivos a llenar

## Transformación de datos

Hasta ahora nos hemos ocupado de la gestión de los datos que faltan. El filtrado, la limpieza y otras transformaciones son otra clase de operaciones importantes.

### Remover duplicados

Pueden encontrarse filas duplicadas en un DataFrame por cualquier número de razones. He aquí un ejemplo:

In [30]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2":[1, 1, 2, 3, 3, 4, 4]})

In [31]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


El método DataFrame `duplicated` devuelve una serie booleana que indica si cada fila es un duplicado (sus valores de columna son exactamente iguales a los de una fila anterior) o no:

In [32]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

En relación con esto, `drop_duplicates` devuelve un DataFrame con filas en las que se ha filtrado False el array duplicado:

In [33]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Ambos métodos consideran por defecto todas las columnas; alternativamente, puede especificar cualquier subconjunto de ellas para detectar duplicados. Supongamos que tenemos una columna adicional de valores y queremos filtrar los duplicados basándonos sólo en la columna "k1":

In [34]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [35]:
# Añadimos una tercera columna
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [36]:
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` y `drop_duplicates` mantienen por defecto la primera combinación de valores observada. Si se pasa `keep="last"` se devolverá la última.  
El argumento `keep` puede tomar tres valores:

`"first"`: (por defecto) Mantiene la primera aparición de una fila duplicada y elimina las subsecuentes.  

`"last"`: Mantiene la última aparición de una fila duplicada y elimina las anteriores.  

`"False"`: Elimina todas las filas duplicadas, no conservando ninguna.

In [37]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [38]:
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transformación de datos mediante una `Function` o `Mapping`

Para muchos conjuntos de datos, es posible que desee realizar alguna transformación basada en los valores de un array, Serie o columna de un DataFrame. Considere los siguientes datos hipotéticos recogidos sobre varios tipos de jamón:

In [39]:
data = pd.DataFrame({"food": ["bacon", "pulled pork",
                              "bacon","pastrami", "corned beef",
                              "bacon", "pastrami", "honey ham",
                              "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Supongamos que queremos añadir una columna que indique el tipo de animal del que procede cada alimento. Escribamos una correspondencia entre cada tipo de carne y el tipo de animal:

In [40]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

El método `map` de una serie acepta una función u objeto de tipo diccionario que contenga un mapeo para realizar la transformación de los valores:

In [41]:
data["animal"] = data["food"].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


También podríamos haber pasado una función que haga todo el trabajo:

In [42]:
def get_animal(x):
    return meat_to_animal[x]

data["animal"] = data["food"].map(get_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### Sustitución de valores

Rellenar los datos que faltan con el método `fillna` es un caso especial de sustitución de valores más general. Como ya se ha visto, `map` puede utilizarse para modificar un subconjunto de valores de un objeto, tambien `replace` proporciona una forma más sencilla y flexible de hacerlo. Consideremos esta serie:

In [43]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

Los valores `-999` podrían ser valores centinela de datos perdidos. Para reemplazarlos por valores `NA` que pandas entienda, podemos usar `replace`, produciendo una nueva Serie:

In [44]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

Si desea sustituir varios valores a la vez, debe pasar una lista y, a continuación, el valor sustituido:

In [45]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

Para utilizar un sustituto diferente para cada valor, pase una lista de sustitutos:

In [46]:
 data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

El argumento pasado también puede ser un diccionario:

In [47]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

El método `data.replace()` es distinto de `data.str.replace`, que realiza la sustitución de cadenas por elementos. Veremos estos métodos de cadena en Series más adelante.

### Renombrar índices de ejes

Al igual que los valores de una Serie, las etiquetas de los ejes pueden transformarse de forma similar mediante una función o un mapeo (`mapping`) de algún tipo para producir nuevos objetos etiquetados de forma diferente. También puede modificar los ejes in situ sin crear una nueva estructura de datos. He aquí un ejemplo sencillo:

In [48]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])
                    
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Al igual que una Serie, los índices de eje tienen un método `map`:

In [49]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)
#Index(['OHIO', 'COLO', 'NEW '], dtype='object')

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [50]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


You can assign to the `index` attribute, modifying the DataFrame in place:

In [51]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Si desea crear una versión transformada de un conjunto de datos sin modificar el original, un método útil es `rename`:


In [52]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


En particular, renombrar puede utilizarse junto con un objeto tipo diccionario, proporcionando nuevos valores para un subconjunto de las etiquetas de los ejes:

In [53]:
data.rename(index={"OHIO": "Madrid"},
            columns={"three": "tres"})
            

,one,two,tres,four
Madrid,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretización y `binning`

La discretización y el binning son técnicas utilizadas en la preprocesamiento de datos para convertir datos continuos en discretos. Estas técnicas son útiles para reducir la variabilidad, agrupar datos similares y mejorar el rendimiento de algunos algoritmos de aprendizaje automático

### Discretización.  

Es el proceso de convertir datos continuos en datos categóricos dividiendo el rango de valores continuos en intervalos (bins).

### Binning (Agrupación por intervalos)
El `binning` es una técnica de discretización que agrupa los datos en intervalos, o `"bins"`.  
Hay varios métodos para hacer `binning` en pandas, como el binning por frecuencia, el binning por longitud de intervalo y el binning basado en la cuantilación (por ejemplo Cuartil).

Los datos continuos suelen discretizarse o separarse en "intervalos" (bins) para su análisis. Supongamos que se dispone de datos sobre un grupo de personas en un estudio y desea agruparlas en grupos de edad discretos:

In [54]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Vamos a dividirlos en franjas (bins) de 18 a 25, de 26 a 35, de 36 a 60 y, por último, de 61 años en adelante. Para ello, hay que utilizar `pandas.cut`:

In [55]:
bins = [18, 25, 35, 60, 100]

In [56]:
age_categories = pd.cut(ages, bins)

In [57]:
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

El objeto que pandas devuelve es un objeto `Categorical` especial. La salida que se ve describe los bins calculados por `pandas.cut`. Cada bin se identifica por un tipo de valor de intervalo especial (único en pandas) que contiene el límite inferior y superior de cada bin:

In [58]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [59]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [60]:
age_categories.categories[0]

Interval(18, 25, closed='right')

In [61]:
pd.value_counts(age_categories)

C:\Users\juanj\AppData\Local\Temp\ipykernel_24584\3010498523.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(age_categories)


(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

Tenga en cuenta que `pd.value_counts(categories)` son los recuentos bin del resultado de `pandas.cut`.

En la representación de cadena de un intervalo, un paréntesis significa que el lado está abierto (excluyente), mientras que el corchete significa que está cerrado (incluyente). Puede cambiar el lado cerrado pasando `right=False`

In [62]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

Puede anular el etiquetado de contenedores por defecto basado en intervalos pasando una lista o array a la opción `labels`:

In [63]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

Consideremos el caso de unos datos distribuidos uniformemente y cortados en cuartos:

In [64]:
data = np.random.uniform(size=20)
data

array([0.19834812, 0.53076273, 0.35208551, 0.80617213, 0.07788087,
       0.93115619, 0.32682571, 0.73751538, 0.41027641, 0.61318939,
       0.07088802, 0.24386108, 0.16731792, 0.64121504, 0.44593581,
       0.82643679, 0.2275916 , 0.538186  , 0.73839418, 0.8404235 ])

In [65]:
pd.cut(data, 4, precision=2)

[(0.07, 0.29], (0.5, 0.72], (0.29, 0.5], (0.72, 0.93], (0.07, 0.29], ..., (0.72, 0.93], (0.07, 0.29], (0.5, 0.72], (0.72, 0.93], (0.72, 0.93]]
Length: 20
Categories (4, interval[float64, right]): [(0.07, 0.29] < (0.29, 0.5] < (0.5, 0.72] < (0.72, 0.93]]

La opción `precision=2` limita la precisión decimal a dos dígitos distintos de cero.

Una función estrechamente relacionada, `pandas.qcut`, separa los datos basándose en los cuantiles de la muestra. Dependiendo de la distribución de los datos, el uso de `pandas.cut` no siempre resultará en que cada contenedor tenga el mismo número de puntos de datos. Dado que `pandas.qcut` utiliza los cuantiles de la muestra en su lugar, obtendrá intervalos de tamaño más o menos igual:

In [66]:
data = np.random.standard_normal(1 )
data

array([-3.09268346e-01,  8.60592035e-01, -1.85165971e-01, -4.71546927e-01,
        3.18580942e-01, -2.06018529e+00,  3.72494003e-01, -5.69119202e-01,
       -5.04604078e-01, -5.87250433e-01, -2.37701074e-01, -2.92980763e-01,
        2.41436268e+00,  2.18157475e-01, -3.98198873e-01,  1.04274608e+00,
        1.76838058e+00,  1.35027966e+00,  2.46026619e-02,  5.25024303e-01,
        6.71442706e-01, -7.35708038e-01, -1.00038276e+00, -5.18769871e-01,
       -8.88890755e-02,  4.39346501e-01, -1.59845881e-02, -1.61451172e+00,
        9.94131773e-01,  7.00355220e-01,  8.15292370e-01, -5.23127661e-02,
        8.46694004e-01,  5.06151274e-01,  9.69795320e-01,  6.10639750e-02,
       -8.07918762e-01,  6.57095397e-01, -1.20144435e+00,  8.21814785e-01,
       -3.51248424e-01, -6.72352785e-01,  7.86341241e-01,  3.56367869e-02,
        1.92860011e+00,  4.57564391e-01, -6.18631735e-01,  6.32390262e-01,
        7.26868265e-01,  4.10636040e-02, -3.15152220e-02,  1.19819443e+00,
       -8.01919336e-01,  

In [67]:
quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(-0.72, 0.0047], (0.65, 3.02], (-0.72, 0.0047], (-0.72, 0.0047], (0.0047, 0.65], ..., (0.65, 3.02], (-0.72, 0.0047], (-0.72, 0.0047], (0.65, 3.02], (-0.72, 0.0047]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.07, -0.72] < (-0.72, 0.0047] < (0.0047, 0.65] < (0.65, 3.02]]

In [68]:
pd.value_counts(quartiles)

C:\Users\juanj\AppData\Local\Temp\ipykernel_24584\3472704981.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(quartiles)


(-3.07, -0.72]     250
(-0.72, 0.0047]    250
(0.0047, 0.65]     250
(0.65, 3.02]       250
Name: count, dtype: int64

In [69]:
pd.Series(quartiles).value_counts()

(-3.07, -0.72]     250
(-0.72, 0.0047]    250
(0.0047, 0.65]     250
(0.65, 3.02]       250
Name: count, dtype: int64

De forma similar a `pandas.cut`, el usuario puede pasar sus propios cuantiles (números entre 0 y 1, ambos inclusive):

In [70]:
data

array([-3.09268346e-01,  8.60592035e-01, -1.85165971e-01, -4.71546927e-01,
        3.18580942e-01, -2.06018529e+00,  3.72494003e-01, -5.69119202e-01,
       -5.04604078e-01, -5.87250433e-01, -2.37701074e-01, -2.92980763e-01,
        2.41436268e+00,  2.18157475e-01, -3.98198873e-01,  1.04274608e+00,
        1.76838058e+00,  1.35027966e+00,  2.46026619e-02,  5.25024303e-01,
        6.71442706e-01, -7.35708038e-01, -1.00038276e+00, -5.18769871e-01,
       -8.88890755e-02,  4.39346501e-01, -1.59845881e-02, -1.61451172e+00,
        9.94131773e-01,  7.00355220e-01,  8.15292370e-01, -5.23127661e-02,
        8.46694004e-01,  5.06151274e-01,  9.69795320e-01,  6.10639750e-02,
       -8.07918762e-01,  6.57095397e-01, -1.20144435e+00,  8.21814785e-01,
       -3.51248424e-01, -6.72352785e-01,  7.86341241e-01,  3.56367869e-02,
        1.92860011e+00,  4.57564391e-01, -6.18631735e-01,  6.32390262e-01,
        7.26868265e-01,  4.10636040e-02, -3.15152220e-02,  1.19819443e+00,
       -8.01919336e-01,  

In [71]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-3.057, -1.259]     100
(-1.259, 0.00467]    400
(0.00467, 1.203]     400
(1.203, 3.024]       100
Name: count, dtype: int64

El segundo argumento `[0, 0.1, 0.5, 0.9, 1.]` especifica los puntos de corte para los cuantiles.  
Estos puntos de corte son porcentajes que indican cómo se deben dividir los datos.  

`0` corresponde al valor mínimo.  
`0.1` corresponde al percentil 10 (el valor por debajo del cual se encuentra el 10% de los datos).  
`0.5` corresponde al percentil 50 (la mediana).  
`0.9` corresponde al percentil 90 (el valor por debajo del cual se encuentra el 90% de los datos).  
`1.` corresponde al valor máximo.  

### Detección y filtrado de valores atípicos (outliers)

Filtrar o transformar los valores atípicos es en gran medida una cuestión de aplicar operaciones de arrays. Considere un DataFrame con algunos datos distribuidos normalmente:

In [73]:
data = pd.DataFrame(np.random.standard_normal((50, 4)))
data

,0,1,2,3
0,-2.458797,-1.618339,-1.078522,-2.210120
1,-1.090696,-0.088806,0.341295,2.553908
2,-0.164529,0.553024,1.094892,-0.180225
3,-0.200283,0.678017,1.779362,0.245560
4,0.241989,-1.049293,2.754598,0.877807
5,-0.093527,-0.411866,-2.157203,0.615979
6,0.857927,-0.316657,1.753773,-0.662372
7,0.540887,1.115167,-0.232634,-2.060536
8,0.107399,0.298303,2.902416,0.077047
9,1.338249,-1.461597,2.065558,-2.664225


In [74]:
data.describe()

,0,1,2,3
count,50.000000,50.000000,50.000000,50.000000
mean,0.032737,-0.112191,0.115495,0.015036
std,1.068895,0.825793,1.301505,1.135559
min,-2.458797,-2.386643,-2.157203,-2.664225
25%,-0.597799,-0.454269,-0.937925,-0.726940
50%,0.076377,-0.014028,0.073883,-0.058569
75%,0.572114,0.397309,0.841862,0.818971
max,2.580425,1.765616,2.902416,2.587359


- `count`: El número de observaciones en cada columna. En este caso, todas las columnas tienen 1000 observaciones.  
- `mean`: La media de los valores en cada columna. Dado que los datos se generaron a partir de una distribución normal estándar, las medias están cerca de 0.  
- `std`:La desviación estándar de los valores en cada columna. Esto mide la dispersión de los datos alrededor de la media. La desviación estándar está cerca de 1, que es lo esperado para una distribución normal estándar.  
- `min`:El valor mínimo en cada columna. Estos valores indican los puntos más bajos en la distribución de datos.  
- `25%` (primer cuartil):El primer cuartil de los datos, que es el valor por debajo del cual se encuentra el 25% de los datos. Indica el límite inferior del 50% central de los datos.
- `50%` (mediana):La mediana de los datos, que es el valor por debajo del cual se encuentra el 50% de los datos. Es el punto medio de la distribución.  
- `75%` (tercer cuartil):El tercer cuartil de los datos, que es el valor por debajo del cual se encuentra el 75% de los datos. Indica el límite superior del 50% central de los datos.  
- `max`:El valor máximo en cada columna. Estos valores indican los puntos más altos en la distribución de datos.

Supongamos que quiere encontrar valores en una de las columnas que superen 3 en valor absoluto:

In [75]:
col = data[2]
col

0    -1.078522
1     0.341295
2     1.094892
3     1.779362
4     2.754598
5    -2.157203
6     1.753773
7    -0.232634
8     2.902416
9     2.065558
10    0.311627
11    0.268414
12    0.491597
13    1.496216
14    2.241452
15   -0.802999
16   -0.959702
17    1.181908
18   -0.231369
19   -1.778350
20    0.515389
21   -1.002495
22   -1.185100
23    2.526293
24   -1.952929
25    0.927903
26   -0.954203
27    0.884968
28    0.702756
29    0.348981
30    0.617873
31   -0.359104
32    0.712545
33   -0.837065
34    0.127185
35   -0.964689
36    0.679781
37   -1.828735
38    2.062498
39   -1.941975
40    0.575776
41   -0.541421
42   -0.889091
43    0.020580
44   -1.199905
45   -0.513958
46    0.015884
47   -0.819653
48   -0.063149
49   -1.332524
Name: 2, dtype: float64

In [76]:
col[col.abs() > 2]

4     2.754598
5    -2.157203
8     2.902416
9     2.065558
14    2.241452
23    2.526293
38    2.062498
Name: 2, dtype: float64

Para seleccionar todas las filas que tengan un valor superior a 3 o -3, puede utilizar el método `any` en un DataFrame booleano:

In [77]:
data[(data.abs() > 2).any(axis="columns")]
# Verifica (o entra) por columnas pero selecciona toda la fila.

,0,1,2,3
0,-2.458797,-1.618339,-1.078522,-2.210120
1,-1.090696,-0.088806,0.341295,2.553908
4,0.241989,-1.049293,2.754598,0.877807
5,-0.093527,-0.411866,-2.157203,0.615979
7,0.540887,1.115167,-0.232634,-2.060536
8,0.107399,0.298303,2.902416,0.077047
9,1.338249,-1.461597,2.065558,-2.664225
14,-0.437187,-0.342934,2.241452,0.733488
17,1.765676,0.016217,1.181908,2.587359
23,0.480213,-0.084948,2.526293,-0.748463


Los paréntesis alrededor de `data.abs() > 3` son necesarios para llamar al método `any` sobre el resultado de la operación de comparación.

Los valores pueden fijarse en función de estos criterios. Aquí está el código para limitar los valores fuera del intervalo de -2 a 2:

In [78]:
data_1 = data[data.abs() > 2] 
data_1

,0,1,2,3
0,-2.458797,NaN,NaN,-2.210120
1,NaN,NaN,NaN,2.553908
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,2.754598,NaN
5,NaN,NaN,-2.157203,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,-2.060536
8,NaN,NaN,2.902416,NaN
9,NaN,NaN,2.065558,-2.664225


In [81]:
data_2 = np.sign(data) * 3
data_2

,0,1,2,3
0,-3.0,-3.0,-3.0,-3.0
1,-3.0,-3.0,3.0,3.0
2,-3.0,3.0,3.0,-3.0
3,-3.0,3.0,3.0,3.0
4,3.0,-3.0,3.0,3.0
5,-3.0,-3.0,-3.0,3.0
6,3.0,-3.0,3.0,-3.0
7,3.0,3.0,-3.0,-3.0
8,3.0,3.0,3.0,3.0
9,3.0,-3.0,3.0,-3.0


In [83]:
data_1.describe()

,0,1,2,3
count,3.000000,1.000000,7.000000,5.000000
mean,0.843432,-2.386643,1.770802,-0.358723
std,2.861104,NaN,1.762663,2.683374
min,-2.458797,-2.386643,-2.157203,-2.664225
25%,-0.025065,-2.386643,2.064028,-2.210120
50%,2.408668,-2.386643,2.241452,-2.060536
75%,2.494546,-2.386643,2.640445,2.553908
max,2.580425,-2.386643,2.902416,2.587359


La sentencia `np.sign(data)` produce valores 1 y -1 en función de si los valores de los datos son positivos o negativos:

In [84]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,-1.0,-1.0
1,-1.0,-1.0,1.0,1.0
2,-1.0,1.0,1.0,-1.0
3,-1.0,1.0,1.0,1.0
4,1.0,-1.0,1.0,1.0


### Permutación y muestreo aleatorio

Es posible permutar (reordenar aleatoriamente) una Serie o las filas de un DataFrame usando la función `numpy.random.permutation`. Llamar a `permutation` con la longitud del eje que se desea permutar produce un array de enteros que indican el nuevo ordenamiento:

In [213]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [214]:
sampler = np.random.permutation(5)
sampler

array([1, 4, 3, 2, 0])

Este array puede utilizarse entonces en la indexación basada en `iloc` o en la función equivalente `take` equivalente:

In [215]:
df.take(sampler)

,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
4,28,29,30,31,32,33,34
3,21,22,23,24,25,26,27
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6


In [216]:
df.iloc[sampler]

,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
4,28,29,30,31,32,33,34
3,21,22,23,24,25,26,27
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6


Invocando `take` con `axis="columns"`, también podríamos seleccionar una permutación de las columnas:

In [217]:
column_sampler = np.random.permutation(7)
column_sampler

array([1, 3, 4, 5, 0, 6, 2])

In [218]:
df.take(column_sampler, axis="columns")

,1,3,4,5,0,6,2
0,1,3,4,5,0,6,2
1,8,10,11,12,7,13,9
2,15,17,18,19,14,20,16
3,22,24,25,26,21,27,23
4,29,31,32,33,28,34,30


Para seleccionar un subconjunto aleatorio (ramdom subset) sin reemplazo (la misma fila no puede aparecer dos veces), puede utilizar el método `sample` en Series y DataFrame:

In [219]:
df.sample(n=3)

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
2,14,15,16,17,18,19,20
4,28,29,30,31,32,33,34


Para generar una muestra con reemplazo (para permitir elecciones repetidas), pase `replace=True` a `sample`:

In [220]:
choices = pd.Series([5, 7, -1, 6, 4])
choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

In [221]:
choices.sample(n=10, replace=True)

0    5
1    7
1    7
0    5
3    6
4    4
2   -1
3    6
1    7
0    5
dtype: int64

### Cálculo de indicadores/variables ficticias (dummy)

Otro tipo de transformación para modelado estadístico o aplicaciones de aprendizaje automático es convertir una variable categórica en un array de dummies o indicadores. Si una columna en un DataFrame tiene k valores distintos, se derivaría un array o DataFrame con k columnas que contengan todos los 1s y 0s. Pandas tiene una función `pandas.get_dummies` para hacer esto, aunque también podría idear una usted mismo. Veamos un ejemplo de DataFrame:

In [222]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [223]:
pd.get_dummies(df["key"], dtype=float)

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


Aquí se ha pasado `dtype=float` para cambiar el tipo de salida de boolean (el predeterminado en las versiones más recientes de pandas) a coma flotante (floating point).

En algunos casos, es posible que desee añadir un prefijo a las columnas en el DataFrame del indicador, que luego se pueden fusionar con los otros datos. `pandas.get_dummies` tiene un argumento de prefijo para hacer esto:

In [224]:
dummies = pd.get_dummies(df["key"], prefix="key", dtype=float)
dummies

,key_a,key_b,key_c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [225]:
df_with_dummy = df[["data1"]].join(dummies) # .join lo veremos a detalle mas adelante
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


Si una fila de un DataFrame pertenece a varias categorías, tenemos que utilizar un enfoque diferente para crear las variables ficticias. Veamos el conjunto de datos `MovieLens 1M`:

In [226]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table('movies.dat', sep="::",
                       header=None, names=mnames, engine="python")

movies[:10]                      

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Pandas ha implementado un método especial de la serie `str.get_dummies` (los métodos que empiezan por `str`. Se tratan con más detalle más adelante en Manipulación de cadenas) que maneja este escenario de pertenencia a múltiples grupos codificados como una cadena delimitada:

In [227]:
dummies = movies["genres"].str.get_dummies("|")
dummies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3880,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3881,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [228]:
dummies.iloc[:10, :6]

,Action,Adventure,Animation,Children's,Comedy,Crime
0,0,0,1,1,1,0
1,0,1,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
5,1,0,0,0,0,1
6,0,0,0,0,1,0
7,0,1,0,1,0,0
8,1,0,0,0,0,0
9,1,1,0,0,0,0


Entonces, como antes, puedes combinar esto con `movies` añadiendo un `"Genre_"` a los nombres de las columnas en el DataFrame de `dummies` con el método `add_prefix`:

In [229]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))

In [230]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

Nota: Para datos mucho más grandes, este método de construcción de variables indicadoras con pertenencia múltiple no es especialmente rápido. Sería mejor escribir una función de nivel inferior que escriba directamente en una array de NumPy y, a continuación, envolver el resultado en un DataFrame.

Una receta útil para aplicaciones estadísticas es combinar `pandas.get_dummies` con una función de discretización como `pandas.cut`:

In [231]:
np.random.seed(12345) # para que el ejemplo sea repetible

values = np.random.uniform(size=10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [232]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [233]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False


## 2.3 Extension de tipos de datos 

Pandas se construyó originalmente sobre las capacidades presentes en NumPy, una librería de cálculo de arrays utilizada principalmente para trabajar con datos numéricos. Muchos conceptos de pandas, como los datos que faltan, se implementaron utilizando lo que estaba disponible en NumPy mientras se intentaba maximizar la compatibilidad entre las bibliotecas que utilizaban conjuntamente NumPy y pandas.

Basarse en NumPy conllevó una serie de deficiencias, como:

El manejo de datos faltantes para algunos tipos de datos numéricos, como enteros y booleanos, era incompleto. Como resultado, cuando se introducían datos perdidos en tales datos, pandas convertía el tipo de datos a float64 y utilizaba np.nan para representar valores nulos. Esto tenía efectos agravantes al introducir problemas sutiles en muchos algoritmos de pandas.

Los conjuntos de datos con muchos datos de cadenas eran costosos computacionalmente y utilizaban mucha memoria.

Algunos tipos de datos, como intervalos de tiempo, timedeltas, y timestamps con zonas horarias, no podían ser soportados eficientemente sin usar arrays de objetos Python computacionalmente caros.

Más recientemente, pandas ha desarrollado un sistema de tipos de extensión que permite añadir nuevos tipos de datos aunque no estén soportados nativamente por NumPy. Estos nuevos tipos de datos pueden ser tratados como de primera clase junto con los datos procedentes de arrays NumPy.

Veamos un ejemplo en el que creamos una Serie de enteros con un valor perdido:

In [234]:
s = pd.Series([1, 2, 3, None])
s

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

In [235]:
s.dtype

dtype('float64')

Principalmente por razones de retrocompatibilidad, Series utiliza el comportamiento heredado de usar un tipo de datos float64 y np.nan para el valor perdido. Podríamos crear esta Serie en su lugar usando pandas.Int64Dtype:

In [236]:
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())
s

0       1
1       2
2       3
3    <NA>
dtype: Int64

In [237]:
s.isna()

0    False
1    False
2    False
3     True
dtype: bool

In [238]:
s.dtype

Int64Dtype()

La salida `<NA>` indica que falta un valor para un array de tipo extensión. Esto utiliza el valor centinela especial `pandas.NA`:

In [239]:
s[3]

<NA>

In [240]:
s[3] is pd.NA

True

También podríamos haber utilizado la abreviatura `"Int64"` en lugar de `pd.Int64Dtype()` para especificar el tipo. La mayúscula es necesaria, de lo contrario será un tipo sin extensión basado en NumPy:

In [241]:
s = pd.Series([1, 2, 3, None], dtype="Int64")
s

0       1
1       2
2       3
3    <NA>
dtype: Int64

Pandas también tiene un tipo de extensión especializado para datos de cadena que no utiliza arrays de objetos NumPy (requiere la biblioteca pyarrow, que puede que necesite instalar por separado):

In [242]:
s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s

0      one
1      two
2     <NA>
3    three
dtype: string

Estos arrays de cadenas suelen utilizar mucha menos memoria y, con frecuencia, son más eficientes desde el punto de vista computacional para realizar operaciones en grandes conjuntos de datos.

En la siguiente Tabla figura una lista de algunos de los tipos de extensión disponibles. 

**Pandas extension data types**

`BooleanDtype`: Datos booleanos anulables, utilice `"boolean"` al pasarlos como cadena.

`CategoricalDtype`: Tipo de dato categórico, utilice `"category"` cuando pase como cadena

`DatetimeTZDtype`: Datetime with time zone

`Float32Dtype`: Coma flotante anulable 32-bit , use "Float32" cuando pase como cadena.

`Float64Dtype`: Coma flotante anulable de 64 bits, utilice "Float64" al pasar como cadena.

`Int8Dtype`: Entero con signo anulable de 8 bits, utilice "Int8" al pasarlo como cadena.

`Int16Dtype`: Entero con signo anulable de 16 bits, utilice "Int16" al pasarlo como cadena.

`Int32Dtype`: Entero con signo anulable de 32 bits, utilice "Int32" al pasarlo como cadena.

`Int64Dtype`: Entero con signo anulable de 64 bits, utilice "Int64" al pasarlo como cadena.

`UInt8Dtype`: Entero sin signo anulable de 8 bits, utilice "UInt8" al pasarlo como cadena.

`UInt16Dtype`: Entero sin signo anulable de 16 bits, utilice "UInt16" al pasarlo como cadena.

`UInt32Dtype`: Entero sin signo de 32 bits anulable, utilice "UInt32" al pasarlo como cadena.


`UInt64Dtype`: Entero sin signo de 64 bits anulable, use "UInt64" cuando pase como cadena.





Los tipos de extensión pueden pasarse al método Series `astype`, lo que permite convertirlos fácilmente como parte del proceso de limpieza de datos:

In [243]:
df = pd.DataFrame({"A": [1, 2, None, 4],
                   "B": ["one", "two", "three", None],
                   "C": [False, None, False, True]})

df


,A,B,C
0,1.0,one,False
1,2.0,two,None
2,NaN,three,False
3,4.0,None,True


In [244]:
df["A"] = df["A"].astype("Int64")

In [245]:
df["B"] = df["B"].astype("string")

In [246]:
 df["C"] = df["C"].astype("boolean")

In [247]:
df

,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,<NA>,True


## 2.4 Manipulación de cadenas (string)

Python ha sido durante mucho tiempo un lenguaje popular de manipulación de datos en bruto, en parte debido a su facilidad de uso para el procesamiento de cadenas y texto. La mayoría de las operaciones de texto se simplifican con los métodos incorporados en el objeto string. Para la comparación de patrones y manipulaciones de texto más complejas, pueden ser necesarias expresiones regulares. Pandas se suma a la mezcla al permitirle aplicar expresiones de cadena y regulares de forma concisa en arrays enteros de datos, manejando además la molestia de los datos que faltan.

### Métodos de objetos de cadena incorporados en Python

En muchas aplicaciones de manipulación de cadenas y scripts, los métodos de cadena incorporados son suficientes. Por ejemplo, una cadena separada por comas puede dividirse en trozos con split:

In [248]:
val = "a,b,  guido"
val.split(",")

['a', 'b', '  guido']

`split` se combina a menudo con `strip` para recortar los espacios en blanco (incluidos los saltos de línea):

In [249]:
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'guido']

Estas subcadenas (substrings) podrían concatenarse con un delimitador de dos puntos utilizando la suma:

In [250]:
first, second, third = pieces
first + "::" + second + "::" + third

'a::b::guido'

Pero éste no es un método genérico práctico. Una forma más rápida y pitónica es pasar una lista o tupla al método `join` sobre la cadena "::":

In [251]:
"::".join(pieces)

'a::b::guido'

Otros métodos se ocupan de localizar subcadenas. Utilizar la palabra clave `in` de Python es la mejor forma de detectar una subcadena, aunque también se pueden utilizar `index` y `find`:

In [252]:
"guido" in val

True

In [253]:
val.index(",")

1

In [254]:
val.find(":")

-1

Tenga en cuenta que la diferencia entre `find` e `index` es que index lanza una excepción si no se encuentra la cadena (en lugar de devolver -1):

In [255]:
val.index(":")

ValueError: substring not found

Por su parte, `count` devuelve el número de apariciones de una determinada subcadena:

In [256]:
val.count(",")

2

`replace` sustituirá las ocurrencias de un patrón por otro. También se suele utilizar para eliminar patrones, pasando una cadena vacía:

In [257]:
val.replace(",", "::")

'a::b::  guido'

In [258]:
val.replace(",", "")

'ab  guido'

En el siguiente enlace puede encontrar mas [metodos de cadena]('https://www.w3schools.com/python/python_ref_string.asp')

### Expresiones Regulares

Las expresiones regulares proporcionan una forma flexible de buscar o hacer coincidir patrones de cadenas (a menudo más complejos) en el texto. Una expresión regular, comúnmente llamada regex, es una cadena formada según el lenguaje de expresiones regulares. El módulo re incorporado en Python se encarga de aplicar expresiones regulares a las cadenas; aquí daré varios ejemplos de su uso.

Las funciones del módulo `re` se dividen en tres categorías: coincidencia de patrones, sustitución y división. Naturalmente, todas ellas están relacionadas; un `regex` describe un patrón a localizar en el texto, que luego puede utilizarse para muchos fines. Veamos un ejemplo sencillo: supongamos que queremos dividir una cadena con un número variable de caracteres de espacio en blanco (tabuladores, espacios y nuevas líneas).

La expresión regular que describe uno o más caracteres de espacio en blanco es `\s+`:

In [259]:
import re
text = "foo    bar\t baz  \tqux"

re.split(r"\s+", text)

['foo', 'bar', 'baz', 'qux']

Cuando se llama a `re.split(r"\s+", text)`, primero se compila la expresión regular y luego se llama a su método split sobre el texto pasado. Puede compilar la expresión regular usted mismo con `re.compile`, formando un objeto `regex` reutilizable:

In [260]:
regex = re.compile(r"\s+")

regex.split(text)

['foo', 'bar', 'baz', 'qux']

Si, en cambio, desea obtener una lista de todos los patrones que coincidan con la expresión regular, puede utilizar el método `findall`:

In [261]:
regex.findall(text)

['    ', '\t ', '  \t']

Para evitar escapes no deseados con \ en una expresión regular, utilice literales de cadena sin procesar como `r"C:\x"` en lugar del equivalente `"C:\\x"`.

La creación de un objeto regex con `re.compile` es muy recomendable si se pretende aplicar la misma expresión a muchas cadenas; de este modo se ahorrarán ciclos de CPU.

`match` y `search` están estrechamente relacionados con `findall`. Mientras que `findall` devuelve todas las coincidencias de una cadena, `search` sólo devuelve la primera coincidencia. Más rígidamente, `match` sólo encuentra coincidencias al principio de la cadena. Como ejemplo menos trivial, consideremos un bloque de texto y una expresión regular capaz de identificar la mayoría de las direcciones de correo electrónico:

In [262]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com"""

pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

# re.IGNORECASE hace que la expresión regular no distinga 
# entre mayúsculas y minúsculas. 
regex = re.compile(pattern, flags=re.IGNORECASE)



Utilizando `findall` en el texto se obtiene una lista de las direcciones de correo electrónico:

In [263]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

`search` devuelve un objeto coincidente especial para la primera dirección de correo electrónico del texto. Para la regex anterior, el objeto coincidente sólo puede indicarnos la posición inicial y final del patrón en la cadena:

In [264]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [265]:
text[m.start():m.end()]

'dave@google.com'

`regex.match` devuelve `None`, ya que sólo coincidirá si el patrón aparece al principio de la cadena:

In [266]:
print(regex.match(text))

None


De forma similar, `sub` devolverá una nueva cadena con las ocurrencias del patrón sustituidas por una nueva cadena:

In [267]:
print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED


Supongamos que desea encontrar direcciones de correo electrónico y, al mismo tiempo, segmentar cada dirección en sus tres componentes: nombre de usuario, nombre de dominio y sufijo de dominio. Para ello, ponga entre paréntesis las partes del patrón que desea segmentar:

In [268]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"

regex = re.compile(pattern, flags=re.IGNORECASE)

Un objeto match producido por esta regex modificada devuelve una tupla de los componentes del patrón con su método groups:

In [269]:
m = regex.match("wesm@bright.net")

m.groups()

('wesm', 'bright', 'net')

`findall` devuelve una lista de tuplas cuando el patrón tiene grupos:

In [270]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

`sub` también tiene acceso a los grupos de cada coincidencia mediante símbolos especiales como `\1` y `\2`. El símbolo `\1` corresponde al primer grupo coincidente, `\2` corresponde al segundo, y así sucesivamente:

In [271]:
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com


A continuación algunos metodos adicionales de expresiones regulares.

`findall`: Devuelve todos los patrones coincidentes no solapados en una cadena como una lista.

`finditer`: Como findall, pero devuelve un iterador.

`match`: 	Coincide con el patrón al inicio de la cadena y opcionalmente segmenta los componentes del patrón en grupos; si el patrón coincide, devuelve un objeto coincidente, y en caso contrario `None`.

`search`: Escanea la cadena en busca de coincidencias con el patrón, devolviendo un objeto coincidente si es así; a diferencia de match, la coincidencia puede estar en cualquier parte de la cadena en lugar de sólo al principio.

`split`: Rompe la cadena en trozos en cada aparición del patrón.

`sub, subn`: Reemplazar todas (`sub`) o las primeras `n` ocurrencias (`subn`) del patrón en la cadena con la expresión de reemplazo; utilizar los símbolos `\1`, `\2`, ... para referirse a los elementos del grupo de coincidencia en la cadena de reemplazo.

**JUEVES 05/10/2023**

### Funciones de cadena en pandas

Limpiar un conjunto de datos desordenado para su análisis suele requerir mucha manipulación de cadenas. Para complicar las cosas, una columna que contiene cadenas a veces tiene datos que faltan:

In [272]:
data = {"Dave": "dave@google.com",
        "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com",
        "Wes": np.nan}

data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [273]:
data.isna()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

Se pueden aplicar métodos de cadenas y expresiones regulares (pasando una `lambda` u otra función) a cada valor utilizando `data.map`, pero fallará en los valores NA (nulos). Para hacer frente a esto, Series dispone de métodos orientados a arrays para operaciones con cadenas que omiten y propagan los valores NA. Se accede a ellos a través del atributo str de Series; por ejemplo, podríamos comprobar si cada dirección de correo electrónico contiene "gmail" con `str.contains`:

In [131]:
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Tenga en cuenta que el resultado de esta operación tiene un dtype objeto. pandas tiene tipos de extensión que proporcionan un tratamiento especializado de cadenas, enteros y datos booleanos:

In [132]:
data_as_string_ext = data.astype('string')
data_as_string_ext

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string

In [133]:
data_as_string_ext.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

También se pueden utilizar expresiones regulares, junto con otras opciones como IGNORECASE:

In [134]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

Hay un par de formas de recuperar elementos vectorizados. Puede utilizar str.get o indexar en el atributo str:

In [279]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
# La línea .str[0] se utiliza para obtener la primera tupla
# de la lista, que contiene los valores encontrados para el 
# primer grupo de captura en la expresión regular.
matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [282]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

De forma similar, puede cortar cadenas utilizando esta sintaxis:

In [137]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

El método `str.extract` devolverá los grupos capturados de una expresión regular como un DataFrame:

In [138]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


En la siguiente tabla se muestran otros métodos de String en Series

`cat`: Concatenar cadenas por elementos con un delimitador opcional

`contains`: Devuelve una matriz booleana si cada cadena contiene un patrón/regex.

`count`: Contar ocurrencias del patrón

`extract`: Utilice una expresión regular con grupos para extraer una o varias cadenas de una serie de cadenas; el resultado será un DataFrame con una columna por grupo.

`endswith`: Equivale a x.endswith(pattern) para cada elemento

`startswith`: Equivale a x.startswith(pattern) para cada elemento

 `findall`: Calcula la lista de todas las apariciones del patrón/regex para cada cadena.
 
 `get`: Índice en cada elemento (recuperar el i-ésimo elemento)
 
 `isalnum`: Equivalente a str.alnum incorporado
 
 `isalpha`: Equivalente al built-in str.isalpha.
 
 `isdecimal`: Equivalente al built-in str.isdecimal.

`isdigit`: Equivalente al built-in str.isdigit

`islower`: Equivalent to built-in str.islower

`isnumeric`: Equivalent to built-in str.isnumeric.

`isupper`: Equivalent to built-in str.isupper.

`join`: Une las cadenas de cada elemento de la Serie con el separador pasado.

`len`: Calcular la longitud de cada cadena.

`lower, upper` : Convertir casos; equivalente a x.lower() o x.upper() para cada elemento.

`match`: Utiliza re.match con la expresión regular pasada en cada elemento, devolviendo True o False si coincide

`pad`: Añadir espacios en blanco a la izquierda, a la derecha o a ambos lados de las cadenas

`center`: Equivale a pad(side="both")

`replace`: Reemplazar las ocurrencias de un patrón/regex por otra cadena

`slice`: Corta cada cadena de la serie

`split` : Dividir cadenas según delimitador o expresión regular

`strip`: Recorta los espacios en blanco de ambos lados, incluidas las nuevas líneas.

`rstrip`: Recortar los espacios en blanco de la derecha.

`lstrip`: Recortar los espacios en blanco de la izquierda. 

## 2.5 Datos Categóricos

Ya hemos visto funciones como unique y value_counts, que nos permiten extraer los valores distintos de un array y calcular sus frecuencias, respectivamente:

In [139]:
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values               

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [140]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [141]:
pd.value_counts(values)

C:\Users\juanj\AppData\Local\Temp\ipykernel_20236\3297668723.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(values)


apple     6
orange    2
Name: count, dtype: int64

Muchos sistemas de datos (para almacenamiento de datos, cálculo estadístico u otros usos) han desarrollado enfoques especializados para representar datos con valores repetidos para un almacenamiento y cálculo más eficientes. En el almacenamiento de datos, una práctica recomendada consiste en utilizar las denominadas tablas de dimensiones, que contienen los valores distintos y almacenan las observaciones primarias como claves enteras que hacen referencia a la tabla de dimensiones:

In [142]:
values = pd.Series([0, 1, 0, 0] * 2)
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [143]:
dim = pd.Series(['apple', 'orange'])
dim

0     apple
1    orange
dtype: object

Podemos utilizar el método `take` para restaurar la serie original de cadenas:

In [144]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

Esta representación como números enteros se denomina representación categórica o codificada en diccionario. La array de valores distintos puede denominarse categorías, diccionario o niveles de los datos. Aqui utilizaremos los términos categórico y categorías. Los valores enteros que hacen referencia a las categorías se denominan códigos de categoría o simplemente códigos.

La representación por categorías puede mejorar considerablemente el rendimiento de los análisis. También puede realizar transformaciones en las categorías sin modificar los códigos (sus valores enteros). Algunos ejemplos de transformaciones que pueden realizarse con un coste relativamente bajo son:

- Renombrar categorías
- Añadir una nueva categoría sin cambiar el orden o la posición de las categorías existentes

### Extension de tipos de datos categóricos en Pandas

Pandas tiene un tipo de extensión especial `Categorical` para contener datos que utilizan la representación o codificación categórica basada en enteros. Esta es una técnica popular de compresión de datos para datos con muchas ocurrencias de valores similares y puede proporcionar un rendimiento significativamente más rápido con un menor uso de memoria, especialmente para datos de cadenas.

Volvamos al ejemplo de la serie anterior:

In [145]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2


In [146]:
N = len(fruits)
N

8

In [147]:
rng = np.random.default_rng(seed=12345)

In [148]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': rng.integers(3, 15, size=N),
                   'weight': rng.uniform(0, 4, size=N)},
                    columns=['basket_id', 'fruit', 'count', 'weight'])

df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


Aquí, `df['fruta']` es un array de objetos de cadena de Python. Podemos convertirlo en categórico llamando a:

In [149]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

Los valores de `fruit_cat` son ahora una instancia de pandas.Categorical, a la que se puede acceder mediante el atributo .array:

In [150]:
c = fruit_cat.array
type(c)

pandas.core.arrays.categorical.Categorical

El objeto `Categorical` tiene atributos de categorías y códigos:

In [151]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [152]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

Un truco útil para obtener una correspondencia entre códigos y categorías es:

In [153]:
dict(enumerate(c.categories))


{0: 'apple', 1: 'orange'}

Puede convertir una columna DataFrame en categórica asignando el resultado convertido:

In [154]:
df['fruit'] = df['fruit'].astype('category')

In [155]:
df["fruit"]

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

También puede crear `pandas.Categorical` directamente a partir de otros tipos de secuencias de Python:

In [156]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

Si ha obtenido datos categóricos codificados de otra fuente, puede utilizar el constructor alternativo `from_codes`:

In [157]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

In [158]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

A menos que se especifique explícitamente, las conversiones categóricas no asumen un orden específico de las categorías. Por lo tanto, la matriz de categorías puede estar en un orden diferente dependiendo del orden de los datos de entrada. Cuando utilice `from_codes` o cualquiera de los otros constructores, puede indicar que las categorías tienen un orden significativo:

In [159]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

La salida `[foo < bar < baz]` indica que 'foo' precede a 'bar' en la ordenación, y así sucesivamente. Una instancia categórica desordenada puede ordenarse con `as_ordered`:

In [160]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

Por último, los datos categóricos no tienen por qué ser cadenas, aunque sólo he mostrado ejemplos de cadenas. Un array categórico puede consistir en cualquier tipo de valor inmutable.

### Cálculos con datos categorías

El uso de `Categorical` en pandas comparado con la versión no codificada (como un array de cadenas) generalmente se comporta de la misma manera. Algunas partes de pandas, como la función `groupby`, funcionan mejor cuando se trabaja con `categoricals`. También hay algunas funciones que pueden utilizar la bandera `ordered`.

Consideremos algunos datos numéricos aleatorios y utilicemos la función `pandas.qcut binning`. Esto devuelve `pandas.Categorical`; hemos usado `pandas.cut` anteriormente pero hemos pasado por alto los detalles de cómo funcionan los categóricos:

In [161]:
rng = np.random.default_rng(seed=12345)

draws = rng.standard_normal(1000)

draws[:5]

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331])

Calculemos un intervalo de cuartiles de estos datos y extraigamos algunas estadísticas:

In [162]:
bins = pd.qcut(draws, 4)
bins

[(-3.121, -0.675], (0.687, 3.211], (-3.121, -0.675], (-0.675, 0.0134], (-0.675, 0.0134], ..., (0.0134, 0.687], (0.0134, 0.687], (-0.675, 0.0134], (0.0134, 0.687], (-0.675, 0.0134]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] < (0.687, 3.211]]

Aunque son útiles, los cuartiles exactos de la muestra pueden ser menos útiles para elaborar un informe que los nombres de los cuartiles. Podemos conseguirlo con el argumento labels de `qcut`:

In [163]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q1', 'Q4', 'Q1', 'Q2', 'Q2', ..., 'Q3', 'Q3', 'Q2', 'Q3', 'Q2']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [164]:
bins.codes[:10]

array([0, 3, 0, 1, 1, 0, 0, 2, 2, 0], dtype=int8)

La etiqueta categórica `bins` no contiene información sobre los bordes de los bins en los datos, por lo que podemos utilizar `groupby` para extraer algunos estadísticos de resumen:

In [165]:
bins = pd.Series(bins, name='quartile')

In [168]:
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
# el método agg calcula las estadísticas resumidas para cada grupo, 
# en este caso, el número de elementos en cada grupo, 
# el valor mínimo y el valor máximo.
results

C:\Users\juanj\AppData\Local\Temp\ipykernel_20236\2441121649.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  results = (pd.Series(draws)


,quartile,count,min,max
0,Q1,250,-3.119609,-0.678494
1,Q2,250,-0.673305,0.008009
2,Q3,250,0.018753,0.686183
3,Q4,250,0.688282,3.211418


La columna "quartile" del resultado conserva la información categórica original, incluida la ordenación, de los intervalos:

In [169]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

### Mejor rendimiento con categóricos

Como anteriormente se ha dicho que los tipos categóricos pueden mejorar el rendimiento y el uso de memoria, así que veamos algunos ejemplos. Consideremos una serie con 10 millones de elementos y un pequeño número de categorías distintas:

In [170]:
N = 10_000_000

In [172]:
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))
labels

0          foo
1          bar
2          baz
3          qux
4          foo
          ... 
9999995    qux
9999996    foo
9999997    bar
9999998    baz
9999999    qux
Length: 10000000, dtype: object

Ahora convertimos las etiquetas en categóricas:

In [173]:
categories = labels.astype('category')

Ahora observamos que las etiquetas utilizan bastante más memoria que las categorías:

In [175]:
labels.memory_usage(deep=True)

600000128

In [176]:
categories.memory_usage(deep=True)

10000540

La conversión a categoría no es gratuita, por supuesto, pero es un coste único:

In [177]:
%time _ = labels.astype('category')

CPU times: total: 703 ms
Wall time: 1.26 s


Las operaciones `GroupBy` pueden ser significativamente más rápidas con categóricas porque los algoritmos subyacentes utilizan arrays de códigos basada en enteros en lugar de un array  de cadenas. Aquí comparamos el rendimiento de value_counts(), que utiliza internamente la maquinaria `GroupBy`:

In [178]:
 %timeit labels.value_counts()

718 ms ± 72.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [179]:
%timeit categories.value_counts()

115 ms ± 7.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


###  Métodos categóricos

Las series que contienen datos categóricos disponen de varios métodos especiales similares a los métodos de cadena especializados de Series.str. Esto también proporciona un acceso conveniente a las categorías y códigos. Considere la Serie:

In [182]:
 s = pd.Series(['a', 'b', 'c', 'd'] * 2)
    
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

El atributo especial de acceso `cat` proporciona acceso a métodos categóricos:

In [183]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [184]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

Supongamos que sabemos que el conjunto real de categorías para estos datos se extiende más allá de los cuatro valores observados en los datos. Podemos utilizar el método `set_categories` para cambiarlas:

In [185]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

cat_s2 = cat_s.cat.set_categories(actual_categories)

cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

Aunque parezca que los datos no cambian, las nuevas categorías se reflejarán en las operaciones que las utilicen. Por ejemplo, value_counts respeta las categorías, si están presentes:



In [186]:
 cat_s.value_counts()

a    2
b    2
c    2
d    2
Name: count, dtype: int64

In [187]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
Name: count, dtype: int64

En grandes conjuntos de datos, las categorías se utilizan a menudo como una herramienta conveniente para ahorrar memoria y mejorar el rendimiento. Después de filtrar un gran DataFrame o Series, muchas de las categorías pueden no aparecer en los datos. Para ayudar con esto, podemos utilizar el método remove_unused_categories para recortar las categorías no observadas:

In [189]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [190]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

### Creación de variables ficticias para la modelización

Cuando se utilizan herramientas estadísticas o de aprendizaje automático, a menudo se transforman los datos categóricos en variables ficticias, lo que también se conoce como codificación de una sola vez (one-hot encoding). Esto implica crear un DataFrame con una columna para cada categoría distinta; estas columnas contienen 1s para ocurrencias de una categoría dada y 0 en caso contrario.

Consideremos el ejemplo anterior:

In [139]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

Como se ha mencionado anteriormente , la función pandas.get_dummies convierte estos datos categóricos unidimensionales en un DataFrame que contiene la variable ficticia:

In [140]:
pd.get_dummies(cat_s, dtype=float)

,a,b,c,d
0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0
6,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,1.0
